In [ ]:

import pandas as pd
import numpy as np
import requests
from requests import get

In [ ]:

meta = pd.read_csv('C:/Users/Admin/Desktop/Miniproject/Data/movies.csv')
links = pd.read_csv('C:/Users/Admin/Desktop/Miniproject/Data/links.csv')

In [ ]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [ ]:

links = links.drop(columns=('imdbId'))


links.isna().sum()

movieId      0
tmdbId     181
dtype: int64

In [ ]:
links = links.dropna()
links.shape

(57917, 2)

In [ ]:
# Getting all rows that do not have null values in thier tmdbId column
meta = pd.merge(links,meta,on='movieId',how='left')

In [ ]:
meta.head()

,movieId,tmdbId,title,genres
0,1,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,8844.0,Jumanji (1995),Adventure|Children|Fantasy
2,3,15602.0,Grumpier Old Men (1995),Comedy|Romance
3,4,31357.0,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,11862.0,Father of the Bride Part II (1995),Comedy


In [ ]:
meta.shape

(57917, 4)

In [ ]:
# Cleaning the title column
split_values = meta['title'].str.rsplit('(', n=1, expand = True) 
meta['Title'] = split_values[0] 
meta['Year'] = split_values[1] 
meta['Year'] = meta.Year.str.replace(')','') 
meta['Title'] = meta['Title'].str.replace(r"\(.*\)", "") 

In [ ]:
meta.shape

(57917, 6)

In [ ]:

meta_copy = meta[meta['Title'].str.contains(", The ")]
meta_copy['Title'] = 'The ' + meta_copy['Title'].str.replace(', The ', '')
meta.update(meta[['movieId', 'genres', 'Year']].merge(meta_copy, 'left'))
meta.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,movieId,tmdbId,title,genres,Title,Year
0,1,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,8844.0,Jumanji (1995),Adventure|Children|Fantasy,Jumanji,1995
2,3,15602.0,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995
3,4,31357.0,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,11862.0,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II,1995


In [ ]:
meta.shape

(57917, 6)

In [ ]:

meta_an = meta[meta['Title'].str.contains(", An ")]
meta_an['Title'] = 'An ' + meta_an['Title'].str.replace(', An ', '')
meta.update(meta[['movieId', 'genres', 'Year']].merge(meta_an, 'left'))
meta.shape

In [ ]:

meta_a = meta[meta['Title'].str.contains(", A ")]
meta_a['Title'] = 'A ' + meta_a['Title'].str.replace(', A ', '')
meta.update(meta[['movieId', 'genres', 'Year']].merge(meta_a, 'left'))
meta.shape

In [ ]:

meta_les = meta[meta['Title'].str.contains(", Les ")]
meta_les['Title'] = 'Les ' + meta_les['Title'].str.replace(', Les ', '')
meta.update(meta[['movieId', 'genres', 'Year']].merge(meta_les, 'left'))
meta.shape

In [ ]:

meta_le = meta[meta['Title'].str.contains(", Le ")]
meta_le['Title'] = 'Le ' + meta_le['Title'].str.replace(', Le ', '')
meta.update(meta[['movieId', 'genres', 'Year']].merge(meta_le, 'left'))
meta.shape

In [ ]:

meta_la = meta[meta['Title'].str.contains(", La ")]
meta_la['Title'] = 'La ' + meta_la['Title'].str.replace(', La ', '')
meta.update(meta[['movieId', 'genres', 'Year']].merge(meta_la, 'left'))
meta.shape

In [ ]:

meta_el = meta[meta['Title'].str.contains(", El ")]
meta_el['Title'] = 'El ' + meta_el['Title'].str.replace(', El ', '')
meta.update(meta[['movieId', 'genres', 'Year']].merge(meta_el, 'left'))
meta['Title'] = meta['Title'].str.strip()
meta.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(57917, 6)

In [ ]:
meta.to_csv('/content/drive/My Drive/Colab Notebooks/Notebooks/Hybrid Movie Recommender System /cleaned_movies.csv')

In [ ]:
meta.head()

,movieId,tmdbId,title,genres,Title,Year
0,1,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,8844.0,Jumanji (1995),Adventure|Children|Fantasy,Jumanji,1995
2,3,15602.0,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995
3,4,31357.0,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,11862.0,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II,1995


In [ ]:

pip install tmdbv3api

In [ ]:
# Importing tmdbv3api 
from tmdbv3api import TMDb
import json 


tmdb = TMDb()
tmdb.api_key = 'd8a9368c0b37d91a60345f5fc99f00d4'

In [ ]:

from tmdbv3api import Movie
tmdb_movie = Movie()

In [ ]:
result = tmdb_movie.details(862.0)
print(result)
movie_id = result.id
print(movie_id)

Toy Story
862


In [ ]:

error_movies = []
new_movies_Id = []
for x in meta['tmdbId']:
  try:
    result = tmdb_movie.details(x)
    #print(result)
    movie_id = result.id
    new_movies_Id.append(movie_id)

  except:
    error_movies.append(result)

In [ ]:
len(new_movies_Id)

57271

In [ ]:
type(new_movies_Id)

list

In [ ]:
df = meta[meta.tmdbId.isin(new_movies_Id)]

In [ ]:
df = df.drop(columns = ['movieId', 'title'])

In [ ]:
df = df[['tmdbId', 'Title', 'Year', 'genres']]

In [ ]:
df['tmdbId'] = df['tmdbId'].apply(np.int64)

In [ ]:
df.head()

,tmdbId,Title,Year,genres
0,862,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy
1,8844,Jumanji,1995,Adventure|Children|Fantasy
2,15602,Grumpier Old Men,1995,Comedy|Romance
3,31357,Waiting to Exhale,1995,Comedy|Drama|Romance
4,11862,Father of the Bride Part II,1995,Comedy


In [ ]:
df.isna().any()

tmdbId    False
Title     False
Year       True
genres    False
dtype: bool

In [ ]:
df.isna().sum()

tmdbId      0
Title       0
Year      284
genres      0
dtype: int64

In [ ]:
df = df.dropna(subset=['Year']) # Dropping all the null values in the Year column

In [ ]:
df.shape

(56987, 4)

In [ ]:
df.isna().sum()

tmdbId    0
Title     0
Year      0
genres    0
dtype: int64

In [ ]:
df.to_csv('C:/Users/Admin/Desktop/Miniproject/Data/cleaned_movies.csv')